In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import pyspark

In [ ]:
pd.read_csv()

### Ejercicio 1
1) Nintendo of America (EEUU) tiene
información de ventas de videojuegos
físicas mensuales totalizadas en EEUU
las cuales se realizan en cadenas de
tiendas de videojuegos en el siguiente
RDD: (id_videojuego, id_tienda, mes,
anio, total_ventas_mensuales).
Por otro lado tenemos un RDD con
información de las tiendas y de su
ubicación (id_tienda, direccion, latitud,
longitud, codigo_postal, estado).

Con esta información escribir un
programa en pySpark para obtener la
tienda que realizó menor cantidad de
ventas en el estado de “Georgia” en todo
el año 2017. (***) (15 pts)

In [35]:
#(id_videojuego, id_tienda, mes, anio, total_ventas_mensuales)
videojuegos = [
    ("v1", "t1", "01", "2017", 10), ("v1", "t1", "03", "2017", 15), ("v1", "t1", "05", "2017", 15),
    ("v1", "t2", "01", "2017", 11), ("v1", "t2", "03", "2017", 23), ("v1", "t2", "05", "2017", 16),
    ("v1", "t3", "01", "2017", 12), ("v1", "t3", "03", "2017", 22), ("v1", "t3", "05", "2017", 17),
    ("v1", "t4", "01", "2017", 13), ("v1", "t4", "03", "2017", 17), ("v1", "t4", "05", "2017", 19),
    ("v1", "t1", "02", "2017", 14), ("v1", "t1", "04", "2017", 11), ("v1", "t1", "06", "2017", 11),
    ("v1", "t2", "02", "2017", 15), ("v1", "t2", "04", "2017", 12), ("v1", "t2", "06", "2017", 20),
    ("v1", "t3", "02", "2017", 11), ("v1", "t3", "04", "2017", 10), ("v1", "t3", "06", "2017", 12),
    ("v1", "t4", "02", "2017", 19), ("v1", "t4", "04", "2017", 40), ("v1", "t4", "06", "2017", 10),
    ("v2", "t1", "01", "2017", 25), ("v2", "t1", "03", "2017", 37), ("v2", "t1", "05", "2017", 12),
    ("v2", "t2", "01", "2017", 25), ("v2", "t2", "03", "2017", 25), ("v2", "t2", "05", "2017", 22),
    ("v2", "t3", "01", "2017", 35), ("v2", "t3", "03", "2017", 22), ("v2", "t3", "05", "2017", 25),
    ("v2", "t4", "01", "2017", 15), ("v2", "t4", "03", "2017", 17), ("v2", "t4", "05", "2017", 26),
    ("v2", "t1", "02", "2017", 12), ("v2", "t1", "04", "2017", 22), ("v2", "t1", "06", "2017", 20),
    ("v2", "t2", "02", "2017", 18), ("v2", "t2", "04", "2017", 21), ("v2", "t2", "06", "2017", 33),
    ("v2", "t3", "02", "2017", 10), ("v2", "t3", "04", "2017", 12), ("v2", "t3", "06", "2017", 12),
    ("v2", "t4", "02", "2017", 16), ("v2", "t4", "04", "2017", 16), ("v2", "t4", "06", "2017", 18)
]
#(id_tienda, direccion, latitud, longitud, codigo_postal, estado)
tiendas = [
    ("t1", "Calle Falsa", "33W", "25S", "14000", "Georgia"),
    ("t2", "Calle Falsa", "33W", "25S", "14000", "Georgia"),
    ("t3", "Calle Falsa", "33W", "25S", "14000", "Georgia"),
    ("t4", "Calle Verdadera", "30W", "20S", "15000", "New York"),
]

rdd_v1 = sc.parallelize(videojuegos)
rdd_t1 = sc.parallelize(tiendas)
rdd_v2 = rdd_v1.filter(lambda x: x[3]=="2017").map(lambda x: (x[1],x[4])) #(id_tienda, ventas_mes)) #solo 2017
rdd_v3 = rdd_v2.reduceByKey(lambda x,y: x+y) #(id_tienda, ventas_2017)
rdd_t2 = rdd_t1.filter(lambda x: x[5]=="Georgia").map(lambda x: (x[0],0)) #(id_tienda,0) #solo Georgia
result = rdd_v3.join(rdd_t2) #(id_tienda, (ventas_2017,0))
result = result.map(lambda x: ((x[0],x[1][0]))) # (id_tienda, ventas_2017)
result = result.reduce(lambda x,y: x if (x[1] < y[1]) else y)
result

('t3', 200)

### Ejercicio 2
2) (****) (15 pts) El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en
Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el siguiente formato: (fecha, id_caso,
descripcion, estado_caso, categoria, latitud, longitud).
Los posibles estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolución. Las
fechas se encuentran en el formato YYYY-MM-DD.
Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para
pedir ayuda del vigilante, Batman. Esta información se encuentra en un Dataframe con el siguiente formato (id_caso,
respuesta), siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él
El sector encargado de las estadísticas oficiales del GCPD quiere con esta información analizar las siguientes
situaciones:
- Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no
participó Batman).
- Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos en los que fue llamado con la
batiseñal, participó en la resolución).
- Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos

In [ ]:
casos_columns = ("fecha", "id_caso", "estado_caso", "categoria")
casos = [
    ("2017-08-23", 1, 2, "hurto"),
    ("2017-08-23", 2, 3, "asesinato"),
    ("2017-08-24", 3, 3, "hurto"),
    ("2017-08-24", 4, 2, "robo"),
    ("2017-08-24", 5, 2, "hurto"),
    ("2017-09-02", 6, 1, "asesinato"),
    ("2017-09-24", 7, 2, "secuestro"),
    ("2017-10-24", 8, 1, "robo"),
    ("2017-12-23", 9, 2, "hurto"),
    ("2017-11-23", 10, 3, "asesinato"),
    ("2017-05-24", 11, 3, "secuestro"),
    ("2017-07-18", 12, 2, "robo"),
    ("2017-04-19", 13, 3, "hurto"),
    ("2017-02-05", 14, 3, "asesinato"),
    ("2017-05-15", 15, 3, "hurto"),
    ("2017-02-11", 16, 1, "robo"),
    ("2017-04-13", 17, 2, "secuestro"),
    ("2017-03-12", 18, 3, "asesinato"),
    ("2017-06-14", 19, 2, "secuestro"),
    ("2017-11-14", 20, 3, "robo"),
    ("2017-11-23", 21, 1, "hurto"),
    ("2017-12-03", 22, 3, "hurto"),
    ("2017-10-14", 23, 2, "secuestro"),
    ("2017-08-14", 24, 2, "robo"),
    ("2017-12-11", 25, 1, "robo"),
    ("2017-12-13", 26, 2, "secuestro"),
    ("2017-12-12", 27, 3, "asesinato"),
    ("2017-12-14", 28, 2, "secuestro"),
    ("2017-12-14", 29, 3, "robo"),
    ("2017-12-23", 30, 1, "hurto"),
    ("2017-12-03", 31, 2, "hurto"),
    ("2017-12-14", 32, 2, "secuestro"),
    ("2017-12-14", 33, 2, "robo"),
    ("2017-08-23", 34, 3, "hurto"),
    ("2017-08-23", 35, 3, "asesinato"),
    ("2017-08-24", 36, 3, "hurto"),
    ("2017-08-24", 37, 2, "robo"),
    ("2017-08-24", 38, 2, "hurto"),
    ("2017-09-02", 39, 1, "asesinato"),
    ("2017-09-24", 40, 2, "secuestro"),
    ("2017-10-24", 41, 2, "robo"),
    ("2017-12-23", 42, 2, "hurto"),
    ("2017-11-23", 43, 2, "asesinato"),
    ("2017-05-24", 44, 3, "secuestro"),
    ("2017-06-18", 45, 2, "robo"),
    ("2017-06-19", 46, 2, "hurto"),
    ("2017-02-05", 47, 3, "asesinato"),
    ("2017-11-15", 48, 3, "hurto"),
    ("2017-02-11", 49, 3, "robo"),
    ("2017-04-13", 50, 2, "secuestro"),
    ("2017-08-12", 51, 3, "asesinato"),
    ("2017-07-14", 52, 2, "secuestro"),
    ("2017-09-14", 53, 3, "robo"),
    ("2017-06-23", 54, 2, "hurto"),
    ("2017-12-03", 55, 2, "hurto"),
    ("2017-03-14", 56, 2, "secuestro"),
    ("2017-02-14", 57, 2, "robo"),
    ("2017-01-11", 58, 1, "robo"),
    ("2017-01-13", 59, 2, "secuestro"),
    ("2017-01-12", 60, 3, "asesinato"),
    ("2017-02-14", 61, 2, "secuestro"),
    ("2017-08-14", 62, 3, "robo"),
    ("2017-10-23", 63, 2, "hurto"),
    ("2017-12-03", 64, 2, "hurto"),
    ("2017-10-14", 65, 2, "secuestro"),
    ("2017-11-14", 66, 2, "robo")
]

casos_df = pd.DataFrame(data=casos,columns=casos_columns)
batman_columns = ("id_caso", "respuesta")
batman = [
    (1,1), (2,0), (3,0),(4,1), (5,0), (6,1), (7,0), (8,0),  (9,1), 
    (10,1), (11,0), (12,1), (13,0), (14,0), (15,1), (16,1), (17,0), (18,1), (19,1),
    (20,0), (21,0), (22,0), (23,0), (24,1), (25,1), (26,0), (27,1), (28,1), (29,0),
    (30,0), (31,0), (32,0), (33,1), (34,1), (35,0), (36,1), (37,1), (38,0), (39,1),
    (40,0), (41,0), (42,1), (43,1), (44,0), (45,0), (46,1), (47,1), (48,0), (49,0), 
    (50,1), (51,0), (52,0), (53,0), (54,0), (55,1), (56,1), (57,1), (58,0), (59,0),
    (60,1), (61,1), (62,1), (63,0), (64,0), (65,1), (66,1)]
batman_df = pd.DataFrame(data=batman,columns=batman_columns)
batman_df.index = batman_df.id_caso
casos_df["batman"] = casos_df.id_caso.map(batman_df.respuesta).astype("int")
sin_batman = casos_df.loc[casos_df.batman==0].groupby(["categoria","estado_caso"]).agg({"estado_caso":"count"}).unstack(fill_value=0)
con_batman = casos_df.loc[casos_df.batman==1].groupby(["categoria","estado_caso"]).agg({"estado_caso":"count"}).unstack(fill_value=0)
sin_batman["suma"] = sin_batman.sum(axis=1)
con_batman["suma"] = con_batman.sum(axis=1)
tasa_sin_batman = sin_batman.estado_caso[2] / sin_batman.suma
tasa_con_batman = con_batman.estado_caso[2] / con_batman.suma
print("tasa_sin_batman\n",tasa_sin_batman,"\n\ntasa_con_batman\n",tasa_con_batman)
casos_df['fecha'] = pd.DatetimeIndex(casos_df['fecha'])
casos_df['year'] = casos_df.fecha.dt.year
casos_df['month'] = casos_df.fecha.dt.month
por_mes = casos_df.loc[casos_df.year==2017].groupby(["month"]).agg({"batman":"sum"})
por_mes.nlargest(1,columns="batman")

### Ejercicio 7
7) El Fiscal de Distrito Harvey Dent no esta convencido de que la irrupción de
Batman en ciudad Gótica le haya significado a la población y al departamento
de policía una mejora en la lucha contra el crimen.
Es tu misión ayudar al Comisionado Gordon planteando una visualización para
demostrar a lo largo del tiempo como fue evolucionando la lucha contra el
crimen a partir de la participación de Batman, y el valor que le brinda al GCPD
su ayuda. (***) (15 pts)

In [ ]:
casos_df.index = casos_df.id_caso
casos_df = casos_df.drop(columns=["id_caso"])

In [ ]:
casos_sin_batman_si_resueltos = casos_df.loc[(casos_df.batman == 0) & (casos_df.estado_caso == 2)]
casos_sin_batman_no_resueltos = casos_df.loc[(casos_df.batman == 0) & (casos_df.estado_caso == 3)]
casos_con_batman_si_resueltos = casos_df.loc[(casos_df.batman == 1) & (casos_df.estado_caso == 2)]
casos_con_batman_no_resueltos = casos_df.loc[(casos_df.batman == 1) & (casos_df.estado_caso == 3)]

In [ ]:
agrupados = casos_df.groupby(["year","month","batman","estado_caso"]).agg({"estado_caso":"count"}).unstack(fill_value=0)
casos_con_batman_no_resueltos

In [ ]:
meses = range(1,13)
columns = ["casos_sin_batman_si_resueltos","casos_con_batman_si_resueltos","casos_con_batman_no_resueltos","casos_sin_batman_no_resueltos"]
resultado = pd.DataFrame(index=meses,columns=columns).fillna(0)

In [ ]:
resultado.casos_sin_batman_no_resueltos = resultado.index.map(casos_sin_batman_no_resueltos.groupby(["month"]).agg({"month":"count"}).month).fillna(0).astype("int")
resultado.casos_sin_batman_si_resueltos = resultado.index.map(casos_sin_batman_si_resueltos.groupby(["month"]).agg({"month":"count"}).month).fillna(0).astype("int")
resultado.casos_con_batman_si_resueltos = resultado.index.map(casos_con_batman_si_resueltos.groupby(["month"]).agg({"month":"count"}).month).fillna(0).astype("int")
resultado.casos_con_batman_no_resueltos = resultado.index.map(casos_con_batman_no_resueltos.groupby(["month"]).agg({"month":"count"}).month).fillna(0).astype("int")

In [ ]:
colors = {
    "casos_sin_batman_no_resueltos":"#FF2D00",
    "casos_sin_batman_si_resueltos":"#0BB500",
    "casos_con_batman_si_resueltos":"#6AFF61",
    "casos_con_batman_no_resueltos":"#FF8300"
}
plot_params = {
    "title": 'Crímenes a lo largo del Año',
    "figsize": (12,6),
}
plot = resultado.plot.bar(stacked=True, color = [colors.get(x) for x in resultado.columns], **plot_params)